In [ ]:
# default_exp core

# Data Dev Tools

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

from sqlalchemy import create_engine
import hashlib
import os.path
import pandas as pd

class Database:
    # http://docs.sqlalchemy.org/en/latest/core/engines.html
    """
    Reference Object for Sql Alchemy Engine 
    
    dbtype: one of ['sqlite','pymysql','snowflake']
    kwargs:
        sqlite 
            - path=<path-to-database file>
        pymysql
            - username,password,hostname,database
        snowflake
            - username,password,account,database,schema,warehouse,role
        
    """
    DB_ENGINES = {
        'sqlite': 'sqlite:////{path}',
        'pymysql': 'mysql+pymysql://{username}:{password}@{hostname}/{database}?charset=UTF8MB4',
        'snowflake':"snowflake://{username}:{password}@{account}/{database}/{schema}?warehouse={warehouse}&role={role}",
    }

    # Main DB Connection Ref Obj
    engine = None
    def __init__(self, dbtype,Base=None,Meta=None,**kwargs):
        dbtype = dbtype.lower()
        if dbtype in self.DB_ENGINES.keys():
            engine_url = self.DB_ENGINES[dbtype].format(**kwargs)
            self.engine = create_engine(engine_url)
            print(self.engine)
            
            if Base:
                self.base = Base
            if Meta:
                self.meta = Meta
        else:
            print("DBType is not found in DB_ENGINE")

In [ ]:
#hide
from dotenv import load_dotenv
from os import environ
load_dotenv('.env')

In [ ]:
mlb_db = Database(
    dbtype='pymysql',
    username=environ.get('db_user'),
    password=environ.get('db_password'),
    hostname=environ.get('db_host'),
    database='MLB'
)

In [ ]:
#export 

def cached_query(self,sql_query, parameters=None):
    """
    Method to query data from `Database` Ref Obj and return pandas dataframe
    Parameters
    ----------
    sql_query : str
        saved SQL query
    parameters : dict, optional
        populates named placeholders in query template. 

    Returns
    -------
    df_raw : DataFrame
        Pandas DataFrame with raw data resulting from query
    """
    if parameters:
        sql_query = sql_query.format(**parameters)
    
    # Hash the query
    query_hash = hashlib.sha1(sql_query.encode()).hexdigest()

    # Create the filepath
    file_path = os.path.join("_cache","{}.csv".format(query_hash))

    # Read the file or execute query 
    if os.path.exists(file_path):
        df_raw = pd.read_csv(file_path)
    else:
        with self.engine.connect() as connection:
            try:
                df_raw = pd.read_sql(sql_query, con=connection, params=parameters)
            except (KeyboardInterrupt, SystemExit):
                connecton.close()
            finally:
                connection.close()
        if not os.path.isdir("_cache"):
            os.makedirs("_cache")
        df_raw.to_csv(file_path, index=False)

    return df_raw
Database.cached_query = cached_query

In [ ]:
show_doc(cached_query)

In [ ]:
mlb_db.engine.table_names()

In [ ]:
with mlb_db.engine.connect() as conn:
    matchups = pd.read_sql('select * from matchups limit 10000',conn)
matchups

In [ ]:
q = """
select 
    *
from 
    plays
limit
    100
"""

In [ ]:
plays = mlb_db.cached_query(q)
plays.head()

In [ ]:
plays['result_event'].value_counts()

In [ ]:
example_query = "SELECT * FROM pitches {WHERE} limit 100;"

In [ ]:
mlb_db.cached_query(
    example_query,parameters={"WHERE":"WHERE details_call_code = 'F'"})

In [ ]:
q = "select distinct(details_call_description) result, count(*) as count from pitches group by details_call_description"
mlb_db.cached_query(q)

In [ ]:
with mlb_db.engine.connect() as conn:
    pitches = pd.read_sql("select * from pitches limit 100000",conn)
pitches.info()

In [ ]:
pitches.duplicated(subset=['gamePk','atBatIndex','playEndTime','index']).sum()

In [ ]:
pitches[
    (pitches['playId'].duplicated())
    &(pitches['details_call_description']!='Automatic Ball')
].sort_values('startTime')[
    ['details_call_description','startTime','playId']
]

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()